###  Two Box Variable Stoichiometry and Nitrogen Fixation
--- 

In this notebook I will write simulations to explore the role of deviations from Redfield on the nutrient stoichiometry of the ocean and the biota. The starting point will be Toby Tirrel's two box ocean model which predicts that nitrogen fixers cause the ocean to converge to the Redfield ratio. We will initially study the implications of allowing different N:P ratios in Nitrogen Fixers and phytoplankton, and then we will explore increasing the number of boxes.


In [5]:

from scipy.integrate import ode

%pylab inline

import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
matplotlib.use("Agg")
import scipy.io as sio
import scipy.signal
from pylab import *

asin=np.arcsin
sqrt=np.sqrt
sin=np.sin
cos=np.cos
pi=np.pi
exp=np.exp
tan=np.tan
I=1j
inf=np.inf

hilbert=scipy.signal.hilbert


Populating the interactive namespace from numpy and matplotlib


/usr/lib/pymodules/python2.7/matplotlib/__init__.py:1173: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [169]:
### Here are important constants

# Relative sizes of the high and low lattitude boxes

HLSize = .15
LLSize = .85

SD = 500.0*LLSize   # Surface layer depth in meters
DD = 3230.0  # Deep layer depth in meters

HD = 500*HLSize


K = 3.0 # Mixing coefficient in meters/year
KH = 3.0 # Mixing coefficient between high and deep boxes

SR = .95 # Fraction of TPP regenerated above 500m
SF = .002 # Fraction of TPP permanently incorporated into sediments
DR = .048 # Fraction of TPP regenthat is regenerated via denitrification to N2
RP = 2.0e-4 # River input of diserated below 500m
DN = .015 # Fraction of TPP of N solved P in mol P/m^2/year
RN = 6.0e-3 # River input of dissolved N in mol N/m^2/year
AN = 7.5e-3 # Atmospheric input of N in molN/m^2/year
RPhyto = 30.0 # N:P ratio in organic matter from phytoplankton
RPHigh = 30.0 # N:P ratio in high lattitude phytoplankton
RDiatom = 10.0 # N:P ratio in diatoms at any lattitude

RNFixers = 50.0 # N:P ratio in Nitrogen Fixers
muDiatom = 365.0 # Growth rate of high lattitude phytoplankton
muNF = 60.0 # Maximum growth rate of nitrogen fixing bacteria in units of 1/year
muPhyto = 365.0 # Maximum growth rate of phytoplankton in units of 1/year

PH = 3.0e-2 # Half saturation constant for Phosphate in mol P /m^3
NH = 5.0e-1   # Half saturation constant for Nitrate in mol N /m^3

PDiatom = 3.0e-2*10
NDiatom = 5.0e-1*10



MC = 10.0

MCoefficient = 0*365/5e-4   # Mortality in 1/year
MDiatomCoefficient = 0*365/5e-3  # High Lattitude Mortality
MFixerCoefficient = 0*354/5e-4


### Next are the functions that define the right hand side of the system of equations

# We first define the dynamical variables:

# NFixer stands for the population of nitrogen fixers
# PhytoP stands to phytoplankton
# Diatom stands for Diatom
# PS stands for the surface phosphate
# NS stands for surface nitrate
# ND stands for deep nitrate
# PD stands for deep phosphate


###

def dNFixer(NFixer,PS):
    return muNF*PS/(PS+PH)*NFixer - (MFixerCoefficient*NFixer+MC)*NFixer

def dPhytoP(PhytoP,PS,NS):
    return muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP - (MCoefficient*PhytoP+MC)*PhytoP

def dDiatom(Diatom,PS,NS):
    return muDiatom*min([PS/(PS+PDiatom), NS/(NS+NDiatom)])*Diatom-MDiatomCoefficient*Diatom*Diatom-MC*Diatom

def dPS(PS,NS,PD,ND,PhytoP,NFixer,Diatom):
    return -muNF*PS/(PS+PH)*NFixer/RNFixers-muDiatom*min([PS/(PS+PDiatom), NS/(NS+NDiatom)])*Diatom/RDiatom+(MDiatomCoefficient*(SR)*Diatom+MC*SR)*Diatom/RDiatom+(MFixerCoefficient*(SR)*NFixer+MC*SR)*NFixer/RNFixers+(MCoefficient*(SR)*PhytoP + MC*SR)*PhytoP/RPhyto-muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP/RPhyto+K*(PD-PS)/SD+RP*LLSize/SD  

def dPSH(PSH,NSH,PD,ND,PhytoPH,DiatomH):
    return (MCoefficient*SR*PhytoPH+MC*SR)*PhytoPH/RPhyto-muPhyto*min([PSH/(PSH+PH),NSH/(NSH+NH)])*PhytoPH/RPhyto+KH*(PD-PSH)/HD+RP*HLSize/HD+((MDiatomCoefficient*DiatomH+MC)*DiatomH*(SR)/RDiatom-DiatomH*muDiatom*min([NSH/(NSH+NDiatom),PSH/(PSH+PDiatom)])/RDiatom)  



def dNS(PS,NS,PD,ND,PhytoP,NFixer,Diatom):
    return -muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP -muDiatom*min([PS/(PS+PDiatom),NS/(NS+NDiatom)])*Diatom+ (MFixerCoefficient+MC/NFixer)*(SR-0.75*DN)*NFixer*NFixer + (MCoefficient+MC/PhytoP)*(SR-0.75*DN)*PhytoP*PhytoP +(MDiatomCoefficient+MC/Diatom)*(SR-0.75*DN)*Diatom*Diatom +K*(ND-NS)/SD+(RN+AN)*LLSize/SD

def dNSH(PSH,NSH,PD,ND,PhytoPH,Diatom):
    return -muPhyto*min([PSH/(PSH+PH),NSH/(NSH+NH)])*PhytoPH+(MCoefficient*SR*PhytoPH+SR*MC)*PhytoPH-muDiatom*min([PSH/(PSH+PDiatom),NSH/(NSH+NDiatom)])*Diatom+(MCoefficient+MC/PhytoPH)*(SR-0.75*DN)*PhytoPH*PhytoPH+KH*(ND-NSH)/HD+(RN+AN)*HLSize/HD+(MDiatomCoefficient*SR*Diatom+SR*MC)*Diatom+(MDiatomCoefficient+MC/Diatom)*(SR-0.75*DN)*Diatom*Diatom                                   
                                   
    

def dPD(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer,Diatom,DiatomH):
    return (MFixerCoefficient*DR*NFixer+MC*DR)*NFixer/RNFixers*SD/DD+(MCoefficient*DR*PhytoP+DR*MC)*PhytoP/RPhyto*SD/DD+(MDiatomCoefficient*DR*Diatom+DR*MC)*Diatom/RDiatom*SD/DD+(MCoefficient*DR*PhytoPH+MC*DR)*PhytoPH/RPhyto*HD/DD +(MDiatomCoefficient*DR*DiatomH+DR*MC)*DiatomH/RDiatom*HD/DD- K*(PD-PS)/DD- KH*(PD-PSH)/DD  


def dND(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer,Diatom,DiatomH):
    return (MFixerCoefficient+MC/NFixer)*(DR-0.25*DN)*NFixer*NFixer*SD/DD+(MDiatomCoefficient+MC/Diatom)*(DR-0.25*DN)*Diatom*Diatom*SD/DD+(MCoefficient+MC/PhytoP)*(DR-0.25*DN)*PhytoP*PhytoP*SD/DD-K*(ND-NS)/DD-KH*(ND-NSH)/DD+(MCoefficient+MC*PhytoPH)*(DR-0.25*DN)*PhytoPH*PhytoPH*HD/DD+(MDiatomCoefficient+MC/DiatomH)*(DR-0.25*DN)*DiatomH*DiatomH*HD/DD


def dPhytoPHigh(PhytoPH,PSH,NSH):
    return muHL*min([PSH/(PSH+PHHigh),NSH/(NSH+NHHigh)])*PhytoPH - MH*PhytoPH



def dState(t,state):
    PS=state[0]
    PSH=state[1]
    NS=state[2]
    NSH=state[3]
    PD=state[4]
    ND=state[5]
    PhytoP=state[6]
    PhytoPH = state[7]
    NFixer=state[8]
    Diatom=state[9]
    DiatomH=state[10]
    if PhytoP<0:
        PhytoP = 1e-12
        state[6]=1e-12
    if NFixer<0:
        NFixer=1e-12
        state[8]=1e-12
    if PhytoPH<0:
        PhytoPH=1e-12
        state[7]=1e-12
    if Diatom<0:
        Diatom=1e-12
        state[9]=1e-12
    if DiatomH<0:
        DiatomH=1e-12
        state[10]=1e-12
    
    
    dS = np.zeros(11)
    dS[0] = dPS(PS,NS,PD,ND,PhytoP,NFixer,Diatom)
    dS[1] = dPSH(PSH,NSH,PD,ND,PhytoPH,DiatomH)
    dS[2] = dNS(PS,NS,PD,ND,PhytoP,NFixer,Diatom)
    dS[3] = dNSH(PSH,NSH,PD,ND,PhytoPH,DiatomH)
    dS[4] = dPD(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer,Diatom,DiatomH)
    dS[5] = dND(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer,Diatom,DiatomH)
    dS[6] = dPhytoP(PhytoP,PS,NS)
    dS[7] = dPhytoP(PhytoPH,PSH,NSH)
    dS[8] = dNFixer(NFixer,PS)
    dS[9] = dDiatom(Diatom,PS,NS)
    dS[10]= dDiatom(DiatomH,PSH,NSH)
    return dS

###










In [170]:
### In this box we will define functions that determine the equilibrium state of the system


PSEquilibrium = M*PH/(muNF-M)
NSEquilibrium = M*NH/(muPhyto-M)

PhytoPEquilibrium = 1.0/(M*SD)*(RN+AN+RP*RNFixers*(1-SF-DN)/SF)/(SF+DN+RNFixers/RPhyto*(1-SF-DN))
NFixerEquilibrium = 1.0/(M*SD)*(RN+AN-RPhyto*RP*(SF+DN)/SF)/(DN+SF-1-RPhyto/RNFixers*(SF+DN))

PDEquilibrium = PSEquilibrium+M*DR*SD/K*(NFixerEquilibrium/RNFixers+PhytoPEquilibrium/RPhyto)
NDEquilibrium = NSEquilibrium+M*(DR-0.25*DN)*SD/K*(NFixerEquilibrium+PhytoPEquilibrium)

RS = NSEquilibrium/PSEquilibrium
RD = NDEquilibrium/PDEquilibrium



NameError: name 'M' is not defined

In [3]:
vecStart=np.array([PSEquilibrium,PSEquilibrium,NSEquilibrium,NSEquilibrium,PDEquilibrium,NDEquilibrium,PhytoPEquilibrium,PhytoPEquilibrium,NFixerEquilibrium,1e-6,1e-76])

NameError: name 'np' is not defined

In [24]:
vecStart= np.array([  1.24431818e-04,   1.24431818e-04,   2.00000000e-03,
         2.00000000e-03,   1.72443182e-03,   2.56000000e-02,
         5.11297341e-05,   5.11297341e-05,   4.41579371e-07, 1e-7,1e-7])

In [6]:
PhytoPEquilibrium

5.112973408541499e-05

In [7]:
(NFixerEquilibrium*RNFixers+PhytoPEquilibrium*RPhyto)/(NFixerEquilibrium+PhytoPEquilibrium)

16.0

In [8]:
dPhytoPHigh(.1,.1,.1)

0.4285714285714288

In [25]:
dState(0,vecStart)


array([  1.16915873e-04,   3.19355816e-04,   3.16970662e-03,
         2.42887325e-02,  -1.71275571e-06,   3.57461704e-05,
        -4.15414052e-03,  -4.15414052e-03,   1.99673311e-05,
         1.04212714e-05,   1.04212714e-05])

In [167]:


rr = ode(dState).set_integrator('lsoda')
rr.set_initial_value(vecStart,0)
rr.t
t1 = 1000
dt = 1e-2
while (rr.successful() and rr.t < t1):
    rr.integrate(rr.t+dt)
    
    
    
    





In [168]:
rr.y

array([  1.60661523e-03,   1.56677838e-03,   2.89786245e-02,
         2.62602622e-02,   1.55810310e-03,   2.56754863e-02,
        -3.71938287e-09,  -3.05149659e-09,  -1.72987770e-08,
        -1.82744099e-08,  -1.82046436e-08])

In [161]:
print rr.y[10]*MDiatomCoefficient-MC
print rr.y[9]*MDiatomCoefficient-MC
print rr.y[8]*MFixerCoefficient-MC
print rr.y[7]*MCoefficient-MC
print rr.y[6]*MCoefficient-MC

-5.00023051702
-5.00024652999
-5.00169896413
8.03734238597
7.8234388935


In [114]:
rr.y[0]/(rr.y[0]+PH)*muDiatom

44.045407971829164

In [165]:
rr.y[5]/rr.y[4]


213.48076049062263

In [124]:
M*DR*SD/K*(PhytoAlt/RPhyto+NFixerAlt/RPhyto)

0.0015728544000000002

In [130]:
PhytoAlt/RPhyto+NFixerAlt/RPhyto-(RP/(M*SD*SF))

-4.648219178082185e-08

In [132]:
RP/(M*SD*SF)

2.7397260273972604e-06

In [133]:
PhytoAlt+NFixerAlt-((1/(M*SD))*(RP*RPhyto*(1-SF-DN)/SF+RN+AN)+(1/(M*SD))*(RP*RPhyto*(SF+DN)-RN-AN))

0.0

In [138]:
1.0/(M*SD)*(RN+AN+RP*(1-SF-DN)/SF)/(SF+DN+RNFixers/RPhyto*(1-SF-DN))

3.063013698630137e-06

In [139]:
PhytoAlt

4.346027397260274e-05

In [141]:
(SF+DN+RNFixers/RPhyto*(1-SF-DN))

1.0

In [149]:
1.0/(M*SD)*(RN+AN+RP*RPhyto*(1-SF-DN)/SF)

4.346027397260274e-05

In [144]:
(1/(M*SD))*(RP*RPhyto*(1-SF-DN)/SF+RN+AN)

4.346027397260274e-05

In [147]:
(RN+AN+RP*(1-SF-DN)/SF)

0.1118

In [148]:
(RP*RPhyto*(1-SF-DN)/SF+RN+AN)

1.5863